
<img align="left" style="padding-right:100px;" src="http://pvexpect.com/Driftsledelse/Dagligvarer/nmbu.png">

  **Digital Driftsledelse - IND310**

 Jesper Ørntoft Frausig, 2021


<a name="3.0"></a>
# **3.0 Linjediagrammer**


- [3.0 Linjediagrammer](#3.0)
- [3.1 Fylt trinndiagram](#3.1)
- [3.2 Linjediagram med konfidensintervall](#3.2)
- [3.3 Linjediagram med lagdelte aggregater](#3.3)
- [3.4 Linjediagram med prosentakse](#3.4)
- [3.5 Linjediagram med punkter](#3.5)
- [3.6 Linjediagram med sekvensgenerator](#3.6)
- [3.7 Linjediagram med varierende størrelse](#3.7)
- [3.8 Linjediagram med flere serier](#3.8)
- [3.9 Skråningsgraf](#3.9)
- [3.10 Trinndiagram](#3.10)


<a name="3.1"></a>
## **3.1 Fylt trinndiagram**

https://altair-viz.github.io/gallery/filled_step_chart.html

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 6.0 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=e9045517f6d8f922a048e1a74b6927ec5c7b4b6292c3d0c2e689be63084e5785
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="1y")
  return hist

In [ ]:
data=stockDownload("Mowi.OL")
data['Dato']=data.index

In [ ]:
import altair as alt


alt.Chart(data).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(
    x='Dato:T',
    y='Close:Q'
)

alt.Chart(...)

<a name="3.2"></a>
## **3.2 Linjediagram med konfidensintervall**

https://altair-viz.github.io/gallery/line_with_ci.html

In [ ]:
import altair as alt
import datetime

data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")&(data['FYLKE']=='01 Finnmark')].groupby(['MÅNED','ÅR','MÅNED_KODE']).sum()
data_selection.reset_index(inplace=True)


data_selection=data_selection.assign(**{'Date':0, 'DAY':1})

for index, måned in enumerate(data_selection.MÅNED):
  
  data_selection.MÅNED[index]=str(måned).replace('DESEMBER','DECEMBER').replace('FEBRUAR','FEBRUARY').replace('JANUAR','JANUARY').replace('JULI','JULY').replace('JUNI','JUNE').replace('MAI','MAY').replace('MARS','MARCH').replace('OKTOBER','OCTOBER')

  data_selection.Date[index] =datetime.datetime.strptime(str(data_selection.ÅR[index])+str(data_selection.MÅNED[index])+str(data_selection.DAY[index]), "%Y%B%d")

data_selection.MÅNED=data_selection.MÅNED.astype(str)
data_selection['timestamp'] = pd.to_datetime(data_selection.Date)

base = alt.Chart(data_selection.drop(['Date','MÅNED'], axis=1)).encode(x="ÅR:O").properties(width=500)

bar = base.mark_line().encode(
    y="mean(BIOMASSE_KG):Q",
    #color="FYLKE:N"
)

band = base.mark_errorband(extent='ci').encode(
    x='ÅR:O',
    y=alt.Y('BIOMASSE_KG:Q', title='BIOMASSE_KG'),
)

alt.layer(bar, band)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


alt.LayerChart(...)

<a name="3.3"></a>
## **3.3 Linjediagram med lagdelte aggregater**

https://altair-viz.github.io/gallery/layer_line_color_rule.html

In [ ]:
#Installere bibliotekker
!pip install yfinance
!pip install pandas-datareader
!pip install yahoofinancials
!pip install dbnomics
!pip install dask[dataframe]
!pip install d6tstack

     |████████████████████████████████| 6.3 MB 7.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=8a7aefea7618e2edaa97db4efeb48b4eedd08cc36a811f5d25e91674ecd8e0d0
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=b59c9ec609afde1ae56b11f1fd9901e76ae5bb4cf8a8de4c6f282c5331aa47ea
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials
     |████████████████████████████████| 125 kB 4.9 MB/s 


In [ ]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="5y")
  return hist

In [ ]:
#Hent aktie symboler
import pandas as pd
url="https://raw.githubusercontent.com/jfrausig/test/master/dagligvarer_ticker.csv" # dk_ticker #trend_bb_4
df=pd.read_csv(url, error_bad_lines=False)
aktier=[]
for index, row in df.iterrows():
    d=row.to_dict()
    aktier.append(d)
aktier=pd.DataFrame(aktier)
aktier['ticker'] = [x.upper() for x in aktier.ticker]




In [ ]:

!mkdir stock_dfs

In [ ]:
import csv
import os
import yfinance as yf
import pandas as pd
import time
start = time.time()

ticker_list = []

ticker_list=list(aktier.ticker)

data = yf.download(
        tickers = ticker_list,
        period = '2y',
        interval = '1wk',
        group_by = 'ticker',
        auto_adjust = False,
        prepost = False,
        threads = True,
        proxy = None
    )

data = data.T

#for ticker in aktier.ticker:
for i, ticker in enumerate(aktier.ticker): 
    aktie=pd.DataFrame(aktier[aktier.ticker==ticker].Virksomhed)['Virksomhed'][i]
    try:
      data.loc[(ticker,),].T.to_csv('stock_dfs/' + aktie + '.csv', sep=',', encoding='utf-8')
    except:
      print(str(aktie)+" try failed")
print('It took', time.time()-start, 'seconds.')

[*********************100%***********************]  12 of 12 completed

3 Failed downloads:
- SALMON.OL: No data found, symbol may be delisted
- SALMAR: No data found, symbol may be delisted
- ORKLA: No data found, symbol may be delisted
It took 1.0174305438995361 seconds.


In [ ]:
#sammenfør virksomheds info
import glob
import d6tstack.combine_csv
import dask.dataframe as dd
cfg_fnames = list(glob.glob('stock_dfs/*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames)

# check columns
print('all equal',c.is_all_equal())
print('')
print(c.is_column_present())

fnames = d6tstack.combine_csv.CombinerCSV(cfg_fnames).to_csv_align(output_dir='test_data/output/')


# consistent format
virksomheds_info = dd.read_csv('test_data/output/*.csv',dtype={'ask': 'float64',
})
aktie_tidsserie=virksomheds_info.compute()

sniffing columns ok
all equal True

                                           Date  Open  High   Low  Close  Adj Close  Volume
file_path                                                                                  
stock_dfs/Andfjord_Salmon.csv              True  True  True  True   True       True    True
stock_dfs/Austevoll Seafood.csv            True  True  True  True   True       True    True
stock_dfs/Bakkafrost.csv                   True  True  True  True   True       True    True
stock_dfs/Grieg Seafood.csv                True  True  True  True   True       True    True
stock_dfs/Lerøy Seafood Group.csv          True  True  True  True   True       True    True
stock_dfs/Mowi.csv                         True  True  True  True   True       True    True
stock_dfs/Norway Royal Salmon.csv          True  True  True  True   True       True    True
stock_dfs/Orkla Group.csv                  True  True  True  True   True       True    True
stock_dfs/SalMar.csv                       T

In [ ]:
import altair as alt
from vega_datasets import data



base = alt.Chart(aktie_tidsserie).properties(width=550)

line = base.mark_line().encode(
    x='Date:O',
    y='Close',
    color='filename'
)

rule = base.mark_rule().encode(
    y='average(Close)',
    color='filename',
    size=alt.value(2)
)

line + rule

alt.LayerChart(...)

<a name="3.4"></a>
## **3.4 Linjediagram med prosentakse**

https://altair-viz.github.io/gallery/line_percent.html

In [ ]:
data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")].groupby(['ÅR']).sum()
data_selection.reset_index(inplace=True)


alt.Chart(data_selection).transform_fold(['UTTAK_STK','DØDFISK_STK','UTKAST_STK','RØMMING_STK','ANDRE_STK']).transform_joinaggregate(
    TotalAmount='sum(BEHFISK_STK):Q',
).transform_calculate(
    PercentOfTotal="datum.value / datum.TotalAmount"
).mark_line().encode(
    alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.2%')),
    x='ÅR:N',
    color='key:N'
)



alt.Chart(...)

<a name="3.5"></a>
## **3.5 Linjediagram med punkter**

https://altair-viz.github.io/gallery/line_chart_with_points.html

In [ ]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="1y")
  return hist

data=stockDownload("Mowi.OL")
data['Dato']=data.index

In [ ]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Dato
Date,,,,,,,,
2020-10-08,167.376696,171.438522,165.989740,171.438522,1546566,0.0,0,2020-10-08
2020-10-09,171.636664,172.379680,170.200170,171.388992,1103884,0.0,0,2020-10-09
2020-10-12,171.190838,171.388973,169.457135,170.398285,986962,0.0,0,2020-10-12
2020-10-13,168.714129,170.002026,167.574829,169.209473,1216348,0.0,0,2020-10-13
2020-10-14,169.159936,169.308533,165.494394,167.574829,1610537,0.0,0,2020-10-14
...,...,...,...,...,...,...,...,...
2021-10-04,222.000000,224.399994,220.399994,222.300003,436395,0.0,0,2021-10-04
2021-10-05,223.399994,224.300003,219.399994,224.100006,720602,0.0,0,2021-10-05
2021-10-06,224.500000,224.699997,218.500000,220.899994,546392,0.0,0,2021-10-06


In [ ]:
alt.Chart(data).mark_line(point=True).encode(
    x='Dato',
    y='High'
)

alt.Chart(...)

<a name="3.6"></a>
## **3.6 Linjediagram med sekvensgenerator**

https://altair-viz.github.io/gallery/line_chart_with_generator.html

<a name="3.7"></a>
## **3.7 Linjediagram med varierende størrelse**

https://altair-viz.github.io/gallery/trail_marker.html

In [ ]:
data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[(data.ARTSID=="LAKS")].groupby(['ÅR']).sum()
data_selection.reset_index(inplace=True)


alt.Chart(data_selection).transform_fold(['UTTAK_STK','DØDFISK_STK','UTKAST_STK','RØMMING_STK','ANDRE_STK']).transform_joinaggregate(
    TotalAmount='sum(BEHFISK_STK):Q',
).transform_calculate(
    PercentOfTotal="datum.value / datum.TotalAmount"
).mark_trail().encode(
    alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.2%')),
    x='ÅR:N',
    color='key:N',
    size='PercentOfTotal:Q'
)



alt.Chart(...)

<a name="3.8"></a>
## **3.8 Linjediagram med flere serier**

https://altair-viz.github.io/gallery/multi_series_line.html

In [ ]:
import altair as alt
from vega_datasets import data



alt.Chart(aktie_tidsserie).mark_line().encode(
    x='Date:O',
    y='Close',
    color='filename',
    strokeDash='filename'
).properties(width=550)



alt.Chart(...)

<a name="3.9"></a>
## **3.9 Skråningsgraf**

https://altair-viz.github.io/gallery/slope_graph.html

In [ ]:
import altair as alt
from vega_datasets import data

source = data.barley()

alt.Chart(source).mark_line().encode(
    x='year:O',
    y='median(yield)',
    color='site'
)

<a name="3.10"></a>
## **3.10 Trinndiagram**

https://altair-viz.github.io/gallery/step_chart.html

In [ ]:
import altair as alt
from vega_datasets import data

source = data.stocks()

alt.Chart(source).mark_line(interpolate='step-after').encode(
    x='date',
    y='price'
).transform_filter(
    alt.datum.symbol == 'GOOG'
)

In [ ]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="12wk")
  return hist

data=stockDownload("Mowi.OL")
data['Dato']=data.index

alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='Dato',
    y='Low'
)

alt.Chart(...)

In [ ]:
"""
import altair as alt
data = pd.read_csv('http://pvexpect.com/Data_forst%C3%A5else_akvakultur_data/akvakultur_data.csv')
data.head(5)

alt.Chart(data).mark_circle(size=60).encode(
    x='Req to dissolved oxygen:Q',
    y='Req to Living space:Q',
    color='Fisk',
    size='Pris',
    tooltip=['Fisk og fiske art']
).interactive()
"""

alt.Chart(...)